Lake Problem without uncertainty

In [16]:
import math
import gym
from gym import spaces
import numpy as np
from scipy.optimize import brentq

class LakeProblem(gym.Env):

    def __init__(self):
        self.b = 0.42  # decay rate for P in lake (0.42 = irreversible)
        self.q = 2.0  # recycling exponent
        self.mean = 0.02  # mean of natural inflows
        self.stdev = 0.0017  # future utility discount rate
        self.delta = 0.98  # standard deviation of natural inflows
        self.alpha = 0.4  # utility from pollution
        self.steps = 20

        high = np.array([5],dtype=np.float32)
        low = np.array([0],dtype=np.float32)

        self.action_space = spaces.Discrete(11)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

    def step(self, action):
        self.time += 1
        if self.time >= self.steps:
            return np.array(0), 0, True, {}
        else:
            t = self.time
            self.pollution[t] = (1 - self.b) * self.pollution[t - 1] + \
                                self.pollution[t - 1] ** self.q / (1 + self.pollution[t - 1] ** self.q) + \
                                action / 100 + self.natural_inflows[t - 1]
            self.benefit[t] = self.alpha * (action / 100) * np.power(self.delta, t)
            reward = self.benefit[t]
            if self.pollution[t] >= self.Pcrit:
                reward -= 1
            return np.array([self.pollution[t]]), reward, False, {}

    def reset(self):
        self.time = 0
        self.Pcrit = brentq(lambda x: x ** self.q / (1 + x ** self.q) - self.b * x, 0.01, 1.5)
        self.natural_inflows = np.random.lognormal(
            math.log(self.mean ** 2 / math.sqrt(self.stdev ** 2 + self.mean ** 2)),
            math.sqrt(math.log(1.0 + self.stdev ** 2 / self.mean ** 2)),
            size=self.steps)
        self.pollution = np.zeros(self.steps)
        self.benefit = np.zeros(self.steps)
        return np.array([self.pollution[0]])

Lake Problem with uncertainty

In [17]:
import math
import gym
from gym import spaces
import numpy as np
from scipy.optimize import brentq

class LakeProblem_uncertain(gym.Env):

    def __init__(self):
        self.b_r = (0.1, 0.45)  # decay rate for P in lake (0.42 = irreversible)
        self.q_r = (2, 4.5)  # recycling exponent
        self.mean_r = (0.01, 0.05)  # mean of natural inflows
        self.stdev_r = (0.001, 0.005)  # future utility discount rate
        self.delta_r = (0.93, 0.99)  # standard deviation of natural inflows
        self.alpha = 0.41  # utility from pollution
        self.steps = 20

        high = np.array([5, 0.45, 4.5, 0.05, 0.005, 0.99],dtype=np.float32)
        low = np.array([0, 0.1, 2, 0.01, 0.001, 0.93],dtype=np.float32)

        self.action_space = spaces.Discrete(11)
        self.observation_space = spaces.Box(low, high, dtype=np.float32)

    def step(self, action):
        self.time += 1
        if self.time >= self.steps:
            return np.array(0), 0, True, {}
        else:
            t = self.time
            self.pollution[t] = (1 - self.b) * self.pollution[t - 1] + \
                                self.pollution[t - 1] ** self.q / (1 + self.pollution[t - 1] ** self.q) + \
                                action / 100 + self.natural_inflows[t - 1]
            self.benefit[t] = self.alpha * (action / 100) * np.power(self.delta, t)
            reward = self.benefit[t]
            if self.pollution[t] >= self.Pcrit:
                reward -= 1
            return np.array((self.pollution[t], self.b, self.q, self.mean, self.stdev, self.delta)), reward, False, {}

    def reset(self):
        self.time = 0
        self.b = np.random.uniform(self.b_r[0], self.b_r[1])
        self.q = np.random.uniform(self.q_r[0], self.q_r[1])
        self.mean = np.random.uniform(self.mean_r[0], self.mean_r[1])
        self.stdev = np.random.uniform(self.stdev_r[0], self.stdev_r[1])
        self.delta = np.random.uniform(self.delta_r[0], self.delta_r[1])
        self.Pcrit = brentq(lambda x: x ** self.q / (1 + x ** self.q) - self.b * x, 0.01, 1.5)
        self.natural_inflows = np.random.lognormal(
            math.log(self.mean ** 2 / math.sqrt(self.stdev ** 2 + self.mean ** 2)),
            math.sqrt(math.log(1.0 + self.stdev ** 2 / self.mean ** 2)),
            size=self.steps)
        self.pollution = np.zeros(self.steps)
        self.benefit = np.zeros(self.steps)
        return np.array((self.pollution[0], self.b, self.q, self.mean, self.stdev, self.delta))

DQN with one network

In [18]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class DeepQNetwork(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
        super(DeepQNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        return actions

class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size,
                 n_acitons, max_mem_size = 100000, eps_end = 0.01, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_acitons)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_counter = 0

        self.Q_eval = DeepQNetwork(self.lr, n_actions=n_acitons, input_dims=input_dims,
                                   fc1_dims=256, fc2_dims=256)

        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done

        self.mem_counter += 1

    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation], dtype=T.float).to(self.Q_eval.device)
            actions = self.Q_eval.forward(state)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        return action

    def learn(self):
        if self.mem_counter < self.batch_size:
            return
        self.Q_eval.optimizer.zero_grad()
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace = False)
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)

        action_batch = self.action_memory[batch]

        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.Q_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0

        q_target = reward_batch + self.gamma * T.max(q_next, dim=1)[0]
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min

DQN with two networks

In [19]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gym

class DeepQNetwork2(nn.Module):
    def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
        super(DeepQNetwork2, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        self.fc3 = nn.Linear(self.fc2_dims, self.n_actions)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        actions = self.fc3(x)
        return actions

class Agent2():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size,
                 n_acitons, max_mem_size = 100000, eps_end = 0.01, eps_dec=5e-4):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.action_space = [i for i in range(n_acitons)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_counter = 0

        self.Q_eval = DeepQNetwork2(self.lr, n_actions=n_acitons, input_dims=input_dims,
                                   fc1_dims=256, fc2_dims=256)
        self.T_eval = DeepQNetwork2(self.lr, n_actions=n_acitons, input_dims=input_dims,
                                   fc1_dims=256, fc2_dims=256)
        self.T_eval.load_state_dict(self.Q_eval.state_dict())
        self.T_eval.eval()
        self.optimizer = optim.Adam(self.Q_eval.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device("cuda:0" if T.cuda.is_available() else "cpu")

        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done

        self.mem_counter += 1

    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation], dtype=T.float).to(self.device)
            actions = self.Q_eval.forward(state)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)
        return action

    def learn(self):
        if self.mem_counter < self.batch_size:
            return
        self.optimizer.zero_grad()
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace = False)
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        state_batch = T.tensor(self.state_memory[batch]).to(self.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.device)

        action_batch = self.action_memory[batch]

        q_eval = self.Q_eval.forward(state_batch)[batch_index, action_batch]
        q_next = self.T_eval.forward(new_state_batch)
        q_next[terminal_batch] = 0.0

        q_target = reward_batch + self.gamma * T.max(q_next, dim=1)[0]
        loss = self.loss(q_target, q_eval).to(self.device)
        loss.backward()
        self.optimizer.step()
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min

Lake Problem without uncertainty + DQN with one network

In [23]:
env = LakeProblem()
agent1 = Agent(gamma=0.99, epsilon = 1.0, batch_size = 64, n_acitons = 11,
              eps_end = 0.01, input_dims = env.observation_space.shape, lr = 0.003)
scores, eps_history = [], []
n_games = 1000

for i in range(n_games):
    score = 0
    done = False
    observation = env.reset()
    actions = []
    while not done:
        action = agent1.choose_action(observation)
        actions.append(action)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent1.store_transition(observation, action, reward, observation_, done)
        agent1.learn()
        observation = observation_
    scores.append(score)
    eps_history.append(agent1.epsilon)
    avg_score = np.mean(scores[-100:])
    print(f"episode {i} \nscore {score} \naverage score {avg_score} \nepsilon {agent1.epsilon}")
    print(actions)
    print()
    if i % 100 == 0:
        print(i)

agent2 = Agent(gamma=0.99, epsilon = 0, batch_size = 64, n_acitons = 11,
              eps_end = 0, input_dims = env.observation_space.shape, lr = 0)
agent2.Q_eval.load_state_dict(agent1.Q_eval.state_dict())
scores, eps_history = [], []
n_games = 10

for i in range(n_games):
    score = 0
    done = False
    observation = env.reset()
    actions = []
    while not done:
        action = agent2.choose_action(observation)
        actions.append(action)
        observation_, reward, done, info = env.step(action)
        score += reward
        observation = observation_
    scores.append(score)
    eps_history.append(agent2.epsilon)
    avg_score = np.mean(scores[-100:])
    print(f"episode {i} \nscore {score} \naverage score {avg_score} \nepsilon {agent2.epsilon}")
    print(actions)
    print()


episode 0 
score -7.606068997909672 
average score -7.606068997909672 
epsilon 1.0
[6, 7, 9, 7, 6, 9, 2, 7, 6, 2, 3, 7, 6, 5, 8, 6, 8, 9, 7, 9]

0
episode 1 
score -4.712732535833674 
average score -6.1594007668716735 
epsilon 1.0
[10, 10, 7, 9, 0, 10, 1, 1, 5, 4, 6, 2, 1, 2, 6, 3, 6, 0, 0, 1]

episode 2 
score -1.7215051937584467 
average score -4.680102242500598 
epsilon 1.0
[9, 0, 5, 2, 3, 10, 0, 9, 8, 9, 0, 4, 0, 6, 4, 2, 1, 4, 8, 8]

episode 3 
score -0.7282220021699168 
average score -3.6921321824179274 
epsilon 0.9915000000000009
[0, 7, 3, 9, 0, 3, 4, 10, 3, 4, 8, 5, 6, 0, 4, 1, 5, 2, 9, 5]

episode 4 
score -3.727437960239798 
average score -3.699193337982302 
epsilon 0.981500000000002
[1, 0, 0, 8, 4, 1, 7, 6, 2, 9, 8, 6, 4, 6, 4, 10, 1, 1, 7, 6]

episode 5 
score 0.25861947269059327 
average score -3.0395578695368193 
epsilon 0.9715000000000031
[7, 4, 5, 5, 4, 3, 0, 3, 1, 5, 7, 3, 2, 6, 0, 8, 3, 7, 6, 9]

episode 6 
score -6.629707967418753 
average score -3.552436454948524 
e

In [21]:
env = LakeProblem_uncertain()
agent1 = Agent(gamma=0.99, epsilon = 1.0, batch_size = 64, n_acitons = 11,
              eps_end = 0.01, input_dims = env.observation_space.shape, lr = 0.003)
scores, eps_history = [], []
n_games = 1000

for i in range(n_games):
    score = 0
    done = False
    observation = env.reset()
    actions = []
    while not done:
        action = agent1.choose_action(observation)
        actions.append(action)
        observation_, reward, done, info = env.step(action)
        score += reward
        agent1.store_transition(observation, action, reward, observation_, done)
        agent1.learn()
        observation = observation_
    scores.append(score)
    eps_history.append(agent1.epsilon)
    avg_score = np.mean(scores[-100:])
    if i % 100 == 0:
        print(i)

agent2 = Agent(gamma=0.99, epsilon = 0, batch_size = 64, n_acitons = 11,
              eps_end = 0, input_dims = env.observation_space.shape, lr = 0)
agent2.Q_eval.load_state_dict(agent1.Q_eval.state_dict())
scores, eps_history = [], []
n_games = 10

for i in range(n_games):
    score = 0
    done = False
    observation = env.reset()
    actions = []
    while not done:
        action = agent2.choose_action(observation)
        actions.append(action)
        observation_, reward, done, info = env.step(action)
        score += reward
        observation = observation_
    scores.append(score)
    eps_history.append(agent2.epsilon)
    avg_score = np.mean(scores[-100:])
    print(f"episode {i} \nscore {score} \naverage score {avg_score} \nepsilon {agent2.epsilon}")
    print(actions)
    print()

0
100
200
300
400
500
600
700
800
900
episode 0 
score -12.885774364714987 
average score -12.885774364714987 
epsilon 0
[4, 4, 7, 0, 0, 0, 10, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

episode 1 
score 0.03094088747810378 
average score -6.427416738618441 
epsilon 0
[4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

episode 2 
score 0.10277311072626344 
average score -4.250686788836873 
epsilon 0
[4, 4, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0, 0, 0, 7, 0]

episode 3 
score 0.06654807499508862 
average score -3.1713780728788827 
epsilon 0
[4, 4, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0]

episode 4 
score 0.13523758409245437 
average score -2.5100549414846154 
epsilon 0
[4, 4, 7, 0, 0, 4, 0, 7, 0, 0, 7, 0, 0, 4, 0, 7, 0, 0, 7, 0]

episode 5 
score 0.062091713798748616 
average score -2.0813638322707213 
epsilon 0
[4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 10, 0, 0, 0, 0, 0, 0]

episode 6 
score 0.12430358380341289 
average score -1.7662684871172736 
epsilon 0
[4, 4,